In [1]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
import coralme
from tqdm import tqdm

In [2]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [3]:
log_format = '%(asctime)s %(message)s' #%(clientip)-15s %(user)-8s
import logging
from coralme.builder.main import ListHandler
log = logging.getLogger()
logging.basicConfig(filename = "./ibdmdb/integration/solving.log", filemode = 'w', level = logging.WARNING, format = log_format)
logging.captureWarnings(True)

In [4]:
from coralme.solver.solver import ME_NLP
import cobra

def get_nlp(model):
    Sf, Se, lb, ub, b, c, cs, atoms, lambdas = model.construct_lp_problem(lambdify=True)
    me_nlp = ME_NLP(Sf, Se,b, c, lb, ub,  cs, atoms, lambdas)
    return me_nlp

def knockout(model,met,me_nlp):
    # met = "RNA_592010.4.peg.10"
    rxns = [r.id for r in model.metabolites.get_by_id(met).reactions if "transcription" in r.id]
    m_idx = model.metabolites.index(met)
    r_idxs = [model.reactions.index(r) for r in rxns ]
    for ri in r_idxs:
        # print(tmp.Sf[(m_idx,ri)])
        me_nlp.Sf[(m_idx,ri)] = 0

def optimize(model,me_nlp,max_mu = 2.8100561374051836, min_mu = 0., maxIter = 100, lambdify = True,
		tolerance = 1e-6, precision = 'quad', verbose = True):
    muopt, xopt, yopt, zopt, basis, stat = me_nlp.bisectmu(
				mumax = max_mu,
				mumin = min_mu,
				maxIter = maxIter,
				tolerance = tolerance,
				precision = precision,
				verbose = verbose)

    if stat == 'optimal':
        #f = sum([ rxn.objective_coefficient * xopt[idx] for idx, rxn in enumerate(self.reactions) ])
        x_primal = xopt[ 0:len(model.reactions) ]   # The remainder are the slacks
        x_dict = { rxn.id : xopt[idx] for idx, rxn in enumerate(model.reactions) }
        #y = pi
        # J = [S; c]
        y_dict = { met.id : yopt[idx] for idx, met in enumerate(model.metabolites) }
        z_dict = { rxn.id : zopt[idx] for idx, rxn in enumerate(model.reactions) }
        #y_dict['linear_objective'] = y[len(y)-1]

        #self.me.solution = Solution(f, x_primal, x_dict, y, y_dict, 'qminos', time_elapsed, status)
        model.solution = cobra.core.Solution(
            objective_value = muopt,
            status = stat,
            fluxes = x_dict, # x_primal is a numpy.array with only fluxes info
            reduced_costs = z_dict,
            shadow_prices = y_dict,
            )
        return True
    else:
        if hasattr(model, 'solution'):
            del model.solution
        return False

In [5]:
def restore(model,met,me_nlp):
    # met = "RNA_592010.4.peg.10"
    rxns = [r.id for r in model.metabolites.get_by_id(met).reactions if "transcription" in r.id]
    m_idx = model.metabolites.index(met)
    r_idxs = [model.reactions.index(r) for r in rxns ]
    for ri in r_idxs:
        # print(tmp.Sf[(m_idx,ri)])
        me_nlp.Sf[(m_idx,ri)] = 1

def printcoeff(model,met,me_nlp):
    # met = "RNA_592010.4.peg.10"
    rxns = [r.id for r in model.metabolites.get_by_id(met).reactions if "transcription" in r.id]
    m_idx = model.metabolites.index(met)
    r_idxs = [model.reactions.index(r) for r in rxns ]
    for ri in r_idxs:
        # print(tmp.Sf[(m_idx,ri)])
        print(me_nlp.Sf[(m_idx,ri)])
        
def get_feasibility(me_nlp, basis=None):
    x_new,y_new,z_new,stat_new,hs_new = me_nlp.solvelp(0.001,basis,'quad')
    return (True,hs_new) if stat_new=="optimal" else (False,hs_new)

In [17]:
# condition = "nonIBD"
condition = "IBD"
# condition = "UC"
# condition = "CD"
fluxdir = "./cases/fluxes/{}".format(condition)
overflowdir = "./cases/overflow/{}".format(condition)
if not os.path.isdir(fluxdir):
    os.makedirs(fluxdir)
if not os.path.isdir(overflowdir):
    os.makedirs(overflowdir)

In [18]:
org = "Eikenella_corrodens_ATCC_23834"
model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
me_nlp = get_nlp(model)

Read LP format model from file /tmp/tmp0f2mhye2.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpq6n2s8sn.lp
Reading time = 0.00 seconds
: 980 rows, 2156 columns, 9364 nonzeros
Read LP format model from file /tmp/tmpfsbs3f36.lp
Reading time = 0.00 seconds
: 1002 rows, 2154 columns, 9194 nonzeros


In [19]:
f,basis = get_feasibility(me_nlp)

In [20]:
f

True

In [21]:
killable = pandas.read_csv("./ibdmdb/integration/{}/{}_killable.txt".format(condition,org),header=None,index_col=0).index.to_list()
for k in killable:
    knockout(model,k,me_nlp)
get_feasibility(me_nlp,basis=basis)

(True, array([0, 3, 3, ..., 0, 0, 3], dtype=int32))

In [22]:
# killable = new_killable

In [23]:
restorelist = []
for k in killable[::-1]:
    restorelist.append(k)
    restore(model,k,me_nlp)
    f,_ = get_feasibility(me_nlp,basis=basis)
    if f:
        print("{} feasible".format(k))
        break
    print("{} not feasible".format(k))
new_killable = [i for i in killable if i not in restorelist]

RNA_EIKCOROL_RS00715 feasible


In [24]:
model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
me_nlp = get_nlp(model)
for k in new_killable:
    knockout(model,k,me_nlp)
get_feasibility(me_nlp,basis=basis)

Read LP format model from file /tmp/tmp5kcgmhve.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpqvltu2s_.lp
Reading time = 0.00 seconds
: 980 rows, 2156 columns, 9364 nonzeros
Read LP format model from file /tmp/tmpc2tyhefe.lp
Reading time = 0.00 seconds
: 1002 rows, 2154 columns, 9194 nonzeros


(True, array([0, 3, 3, ..., 0, 0, 3], dtype=int32))

In [26]:
optimize(model,me_nlp,max_mu=model.solution.objective_value*2)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.0672669568929416	Not feasible
        2	0.0336334784464708	Not feasible
        3	0.0168167392232354	Not feasible
        4	0.0084083696116177	Not feasible
        5	0.0042041848058089	Not feasible
        6	0.0021020924029044	Not feasible
        7	0.0010510462014522	Not feasible
        8	0.0005255231007261	Not feasible
        9	0.0002627615503631	Not feasible
       10	0.0001313807751815	Not feasible
       11	0.0000656903875908	Not feasible
       12	0.0000328451937954	Not feasible
       13	0.0000164225968977	Not feasible
       14	0.0000082112984488	Not feasible
       15	0.0000041056492244	Not feasible
       16	0.0000020528246122	Not feasible
       17	0.0000010264123061	Not feasible
       18	0.0000005132061531	Not feasible


False

In [27]:
get_feasibility(me_nlp)

(True, array([0, 3, 3, ..., 0, 0, 3], dtype=int32))

In [108]:
pandas.DataFrame(columns=new_killable).T.to_csv("./ibdmdb/integration/{}/{}_killable.txt".format(condition,org),header=None)

In [109]:
from overflow import get_overflow
def run(org):
    ListHandler.print_and_log("Loading {}".format(org))
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    me_nlp = get_nlp(model)
    # Run for all conditions
    killable = pandas.read_csv("./ibdmdb/integration/{}/{}_killable.txt".format(condition,org),header=None,index_col=0).index.to_list()
    for k in killable:
        knockout(model,k,me_nlp)
    optimize(model,me_nlp,max_mu=model.solution.objective_value*2)
    sol = model.solution.fluxes["biomass_dilution"] if hasattr(model,"solution") else "infeasible"
    ListHandler.print_and_log("Solution for {} : {}".format(org, sol))
    model.solution.to_frame().to_csv("cases/fluxes/{}/{}.csv".format(condition,org))
    get_overflow(model).to_csv("./cases/overflow/{}/{}.csv".format(condition,org))

In [110]:
run("Ruminococcus_torques_ATCC_27756")

Loading Ruminococcus_torques_ATCC_27756
Read LP format model from file /tmp/tmpjj0w_yy_.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpw04qxh7i.lp
Reading time = 0.00 seconds
: 901 rows, 1988 columns, 8424 nonzeros
Read LP format model from file /tmp/tmpi6kty5eg.lp
Reading time = 0.00 seconds
: 924 rows, 1986 columns, 8304 nonzeros
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.0029157076621025	Not feasible
        2	0.0014578538310513	Not feasible
        3	0.0007289269155256	Not feasible
        4	0.0003644634577628	Optimal
        5	0.0005466951866442	Optimal
        6	0.0006378110510849	Optimal
        7	0.0006833689833053	Optimal
        8	0.0007061479494155	Optimal
        9	0.0007175374324705	Optimal
       10	0.0007232321739981	Optimal
       11	0.0007260795447619	Optimal
       12	0.0007275032301438	Optimal
       13	0.0007282150728347	Optimal
Solution for Ruminococcus_tor

In [7]:
survivors = set(pandas.read_csv("survivors.txt",index_col=0,header=None).index.to_list())
done = set(i.split(".csv")[0] for i in os.listdir("./cases/fluxes/{}/".format(condition)) if "csv" in i)
run_for = survivors - done
len(run_for)

2

In [8]:
run_for

{'Listeria_monocytogenes_Finland_1988', 'Providencia_stuartii_ATCC_25827'}

In [ ]:
NP = min([4,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)

for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

Building (4 threads):   0%|                                                                                 | 0/494 [00:00<?, ?it/s]

Loading Roseburia_intestinalis_L1_82Loading Staphylococcus_lugdunensis_HKU09_01Loading Bacteroides_sp_3_1_40ALoading Bifidobacterium_bifidum_PRL2010



Read LP format model from file /tmp/tmpg0oaps28.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmp5r4m9sh6.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmp4pia0qi9.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpd35ixo4c.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpa2qtx9h0.lp
Reading time = 0.00 seconds
: 990 rows, 2256 columns, 9436 nonzeros
Read LP format model from file /tmp/tmpc9swj8xr.lp
Reading time = 0.00 seconds
: 1170 rows, 2778 columns, 12172 nonzeros
Read LP format model from file /tmp/tmpi18g09fc.lp
Reading time = 0.00 seconds
: 1033 rows, 2286 columns, 9826 nonzeros
Read LP format model from file /tmp/tmp_9l87

Building (4 threads):   0%|▏                                                                    | 1/494 [03:11<26:14:04, 191.57s/it]

Loading Varibaculum_cambriense_DSM_15806
Read LP format model from file /tmp/tmpr3xhtgwn.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmp0lr56o54.lp
Reading time = 0.00 seconds
: 940 rows, 2026 columns, 8774 nonzeros
Read LP format model from file /tmp/tmpwwvr7ez2.lp
Reading time = 0.00 seconds
: 941 rows, 2024 columns, 8604 nonzeros
        7	0.3368773028096805	Not feasible
        2	0.1642574496965015	Not feasible
        8	0.3342859389419138	Not feasible
        6	0.0024839331969461	Optimal
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        9	0.3329902570080304	Not feasible
        7	0.0037258997954192	Not feasible
       10	0.3323424160410887	Not feasible
       11	0.3320184955576179	Optimal
        8	0.0031049164961827	Optimal
       12	0.3321804557993533	Optimal
        9	0.0034154081458010	Not feasible
       13	0.3322614359202210	Optimal
       10	0.0032601623209918	Optimal
       1

Building (4 threads):   0%|▎                                                                     | 2/494 [03:44<13:26:50, 98.39s/it]

...

Loading Actinomyces_turicensis_ACS_279_V_Col4
Read LP format model from file /tmp/tmp3a7a18te.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
        3	0.0719862226952969	Not feasible
        1	0.5316597290723282	Not feasible
Read LP format model from file /tmp/tmp5hmcckft.lp
Reading time = 0.02 seconds
: 1005 rows, 2258 columns, 9706 nonzeros
        4	0.0359931113476485	Not feasible
Read LP format model from file /tmp/tmpu3r4ixnm.lp
Reading time = 0.00 seconds
: 1006 rows, 2256 columns, 9506 nonzeros
        3	0.0687718760877941	Not feasible
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        5	0.0179965556738242	Not feasible
        1	0.2575517202638895	Not feasible
        2	0.2658298645361641	Not feasible
        2	0.1287758601319448	Not feasible
        6	0.0089982778369121	Not feasible
        4	0.0343859380438970	Not feasible
        3	0.0643879300659724	Not feasible
        4	0.0321939650329862	Not feasible
        5	0.0

Building (4 threads):  12%|███████▌                                                          | 57/494 [1:11:20<16:09:26, 133.10s/it]

Loading Eikenella_corrodens_ATCC_23834
Read LP format model from file /tmp/tmpx54n5n2g.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
        9	0.0011247847296140	Optimal
       10	0.0016871770944210	Not feasible
       11	0.0014059809120175	Not feasible
        4	0.0664574661340410	Not feasible
       12	0.0012653828208158	Not feasible
Read LP format model from file /tmp/tmpfnxrx7ob.lp
Reading time = 0.02 seconds
: 980 rows, 2156 columns, 9364 nonzeros
       13	0.0011950837752149	Not feasible
        6	0.0085964845109743	Not feasible
       14	0.0011599342524145	Optimal
       15	0.0011775090138147	Not feasible
Read LP format model from file /tmp/tmpwr8493gh.lp
Reading time = 0.00 seconds
: 1002 rows, 2154 columns, 9194 nonzeros
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
       16	0.0011687216331146	Optimal
       17	0.0011731153234646	Optimal
        1	0.0672669568929416	Not feasible
       18	0.0011753121686396	Not feasible

Building (4 threads):  12%|███████▋                                                          | 58/494 [1:13:06<15:08:17, 124.99s/it]

Loading Bacteroides_faecis_MAJ27
Read LP format model from file /tmp/tmpx0bwm_mu.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
        2	0.0336334784464708	Not feasible
        7	0.0042982422554871	Not feasible
        3	0.0168167392232354	Not feasible
        4	0.0084083696116177	Not feasible
